In [15]:
import sys
sys.path.append("../")

import pandas as pd
import optuna

from run import *

In [16]:
runner = Runner(
    Input,
    Result,
    solver_cmd="solver_cmd",
    solver_version="solver_version",
    database_csv="../log/database.csv",
    input_csv="../log/input.csv",
    log_file="../log/a.log",
    is_maximize=False,
    verbose=0,
)
df = runner.read_database_df()
df = df[df.solver_version.str.startswith("multi-start-")]
df["start_count"] = df.solver_version.str[len("multi-start-"):].astype(int)
df.input_file = df.input_file.str[len("tools/in/"):len("tools/in/0000")].astype(int)
df

,input_file,solver_version,score,duration,d,n,a.sum.mean,best_score,relative_score,start_count
140160,0,multi-start-1,964,2.9051,5,10,777225.800000,332,0.344398,1
140161,1,multi-start-1,71015,2.9097,25,36,984871.680000,50095,0.705414,1
140162,2,multi-start-1,41562,2.9082,25,15,956907.920000,38667,0.930345,1
140163,3,multi-start-1,8333,2.9108,11,42,946477.090909,6895,0.827433,1
140164,4,multi-start-1,9531,2.9052,9,37,972477.333333,8060,0.845662,1
...,...,...,...,...,...,...,...,...,...,...
146655,495,multi-start-13,10676,2.9549,28,36,818769.321429,5884,0.551143,13
146656,496,multi-start-13,1169001,2.9525,47,8,997224.851064,1163001,0.994867,13
146657,497,multi-start-13,2726,2.9525,19,29,779292.526316,1723,0.632062,13
146658,498,multi-start-13,90271,2.9529,35,15,967206.457143,75761,0.839262,13


In [17]:
input_df = pd.read_csv("../log/input.csv")
input_df.input_file = input_df.input_file.str[len("tools/in/"):len("tools/in/0000")].astype(int)
input_df = input_df.set_index("input_file")
inputs = input_df.to_dict(orient="index")

In [18]:
scores = df.groupby(["input_file", "start_count"]).relative_score.mean().to_dict()

In [19]:
from math import pow


def f(
    n: int,
    d: int,
    p1: float,
    p2: float,
    p3: float,
    p4: float,
    p5: float,
    p6: float,
):
    val = n * d / p1
    if val <= p2:
        return p3
    else:
        return min(13, max(1, int(round(p6 - val))))


def objective(trial: optuna.trial.Trial) -> float:
    p1 = trial.suggest_float("p1", low=1.0, high=500.0)
    p2 = trial.suggest_float("p2", low=0.0, high=13.0)
    p3 = trial.suggest_int("p3", low=1, high=13)
    p6 = trial.suggest_float("p6", low=1, high=30)
    score_sum = 0
    for i in range(500):
        n = inputs[i]["n"]
        d = inputs[i]["d"]
        val = f(n, d, p1, p2, p3, p6)
        score_sum += scores[(i, val)]
    return score_sum / 500


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc031.db",
    study_name="tune-start-count-2-2",
    load_if_exists=True,
)
print("start optimize")
study.optimize(objective, n_trials=10000, show_progress_bar=True)
study.best_trial

[I 2024-04-01 15:01:31,106] A new study created in RDB with name: tune-start-count-2-2


start optimize


/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10000 [00:00<?, ?it/s]

[I 2024-04-01 15:01:31,224] Trial 0 finished with value: 0.7973679429762836 and parameters: {'p1': 380.5087122205584, 'p2': 7.156358894488999, 'p3': 1, 'p4': 9.728856066800168, 'p5': 11, 'p6': 3.9607058703379865}. Best is trial 0 with value: 0.7973679429762836.
[I 2024-04-01 15:01:31,324] Trial 1 finished with value: 0.7797664519629962 and parameters: {'p1': 415.6328132868534, 'p2': 7.450868829021059, 'p3': 4, 'p4': 10.82928230469325, 'p5': 10, 'p6': 15.810830679076776}. Best is trial 0 with value: 0.7973679429762836.
[I 2024-04-01 15:01:31,418] Trial 2 finished with value: 0.7796020368351108 and parameters: {'p1': 440.65486654513364, 'p2': 4.074041083226748, 'p3': 1, 'p4': 9.89451379845173, 'p5': 8, 'p6': 7.742328580917974}. Best is trial 0 with value: 0.7973679429762836.
[I 2024-04-01 15:01:31,510] Trial 3 finished with value: 0.7619820000987797 and parameters: {'p1': 426.8482723122538, 'p2': 4.72480761022839, 'p3': 5, 'p4': 8.365823949830892, 'p5': 13, 'p6': 13.942441070357509}. Bes

/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/samplers/_tpe/parzen_estimator.py:193: RuntimeWarning: divide by zero encountered in divide
  coefficient = 1 / z / p_accept


[I 2024-04-01 15:01:32,158] Trial 10 finished with value: 0.7228829534938569 and parameters: {'p1': 153.67748721360286, 'p2': 12.12565309812174, 'p3': 9, 'p4': 12.893892359593238, 'p5': 8, 'p6': 21.93980634619919}. Best is trial 5 with value: 0.8109553384105109.
[I 2024-04-01 15:01:32,258] Trial 11 finished with value: 0.8091508565566113 and parameters: {'p1': 15.95812559585507, 'p2': 10.183834274598189, 'p3': 9, 'p4': 11.25387836213912, 'p5': 1, 'p6': 18.49747819471564}. Best is trial 5 with value: 0.8109553384105109.
[I 2024-04-01 15:01:32,360] Trial 12 finished with value: 0.7307197890694902 and parameters: {'p1': 119.94763642642144, 'p2': 10.63976147233057, 'p3': 10, 'p4': 12.158063157990435, 'p5': 1, 'p6': 29.680044748827598}. Best is trial 5 with value: 0.8109553384105109.

/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:494: RuntimeWarning: invalid value encountered in subtract
  score = log_l - log_g



[I 2024-04-01 15:01:32,460] Trial 13 finished with value: 0.753363395513887 and parameters: {'p1': 100.56551297718855, 'p2': 9.7201745082836, 'p3': 13, 'p4': 11.783328551961434, 'p5': 2, 'p6': 19.704017532661616}. Best is trial 5 with value: 0.8109553384105109.
[I 2024-04-01 15:01:32,555] Trial 14 finished with value: 0.7397437605481118 and parameters: {'p1': 237.26172068506642, 'p2': 12.783186762701419, 'p3': 8, 'p4': 12.98301309527378, 'p5': 6, 'p6': 9.717016103663997}. Best is trial 5 with value: 0.8109553384105109.
[I 2024-04-01 15:01:32,659] Trial 15 finished with value: 0.7944894184607956 and parameters: {'p1': 70.8184683403959, 'p2': 9.967130923619843, 'p3': 10, 'p4': 11.281192863666494, 'p5': 9, 'p6': 18.08732503965945}. Best is trial 5 with value: 0.8109553384105109.
[I 2024-04-01 15:01:32,765] Trial 16 finished with value: 0.7888625105127498 and parameters: {'p1': 212.48599345867345, 'p2': 10.915008646424614, 'p3': 3, 'p4': 12.457337772796969, 'p5': 3, 'p6': 24.3114933715831

KeyboardInterrupt: 

In [ ]:
study = optuna.load_study(
    storage="sqlite:///ahc031.db",
    study_name="tune-start-count-2-1",
)

In [ ]:
study.best_params

{'p1': 227.10122180007895, 'p2': 1.3212813519044897, 'p3': 10, 'p4': 5}